In [11]:
def tokenize(message):
    message = massage.lower()   #轉小寫
    all_words = re.findall("[a-z0-9]+",massege) #提取單詞
    return set(all_words)

def count_words(training_set):
    '''訓練組資料training_set 包含了許多兩兩成對的(massage,is_spam)'''
    counts = defaultdict(lambda:[0,0])
    for message,is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

def word_probabailities(counts,total_spams,total_non_spams,k=0.5):
    '''把word_counts轉換成元組列表
    w , p(w | spam) and p(w | ~spam)'''
    return [(w,
            (spam + k) / (total_spams + 2 * k),
            (non_spam + k) / (total_non_spams + 2 * k))
             for w , (spam, non_spam) in counts.iteritems()]

def spam_probability(word_probs , message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    #對詞表中的每個單詞進行迭代操作
    for word, prob_if_spam , prob_if_not_spam in word_probs:
        #如果郵件有出現相應單詞
        #就把該單詞有出現的相應機率取對數之後再累加進去
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
        #如果郵件中沒出現相應單詞
        #就把該單詞沒出現的相應機率取對數之後,再累加進去
        #單詞沒出現的相應機率,就等於(1 -單詞有出現相應機率)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

class NaiveBayesClassifer:
    def __init__(self,k=0.5):
        self.k = k
        self.word_probs = []
    def train(self,training_set):
        #計算垃圾郵件與非垃圾郵件的數量
        num_spams = len([is_spam
                         for message , is_spam in training_set
                         if is_spam])
        num_non_spams = len(training_set) - num_spams
        
        #運用訓練組數據,執行我們所設計的相關運算
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k)
    def classify(self,message):
        return spam_probability(self.word_probs,message)


In [ ]:
import glob,re
#將路徑修改為實際存檔的位置
path = r"/home/minglu/mlib/spam/*"
data=[]
##glob.glob會送回每一個符合通配(wildcarded)路徑的相應檔案名稱
for fn in glob.glob(path):
    is_spam = "ham" not in fn
    
    with open(fn,'r')as file:
        for line in file:
            if line.startswith('Subject'):
                #移除開頭的'Subject:'字樣,保留其餘的部份
                subject = re.sub(r"^Subject:","",line),strip()
                data.sppend((subject,is_spam))
ramdom.seed(0)
train_data,test_data = split_data(data,0.75)

classifier = NaiveBayesClassifier()
classifier.train(train_data)
#元祖:(主旨,是否確實為垃圾郵件,預測為垃圾郵件的機率)
classified = [(subject,is_spam,classifier.classify(subject))
              for subject , is_spam in test_data]
#假設spam_probability > 0.5 對應的是預測為垃圾郵件
#計算(是否確實為垃圾郵件,是否預測為垃圾郵件)組合的數量
counts = Counter((is_spam,spam_probability > 0.5)
                  for _,is_spam,spam_probability in classified)

#根據垃圾郵件機率
classified.sort(key=lambda row:row[2])
#再非垃圾郵件中,被預測為垃圾郵件最高機率
spammiest_hams = filter(lambda row:not row[1],classified)[-5:]
#再垃圾郵件中,被預測為垃圾郵件的最低的機率者
hammiest_spams = filter(lambda row : row[1],classified)[:5]

def p_spam_given_word(word_prob):
    '''運用貝氏定理計算郵件包含某單詞情況下,該郵件為垃圾郵件機率'''
    #word_prob是由word_probabilities 所生成的某中的一個元組
    word,prob_if_spam,prob_if_not_spam = word_prob
    return prob_if_spam/(prob_if_spam + prob_if_not_spam)
words = sorted(classifier.word_probs,key=p_spam_given_word)

spammiest_words = words[-5:]
hammiest_words = words[:]

In [5]:
!pip install machine_learning

  Could not find a version that satisfies the requirement machine_learning (from versions: )
No matching distribution found for machine_learning
